In [97]:
import urllib.request
import gensim
from gensim.models import word2vec

import nltk.data
import re
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, RegexpTokenizer 

import pandas as pd
from tqdm import tqdm 

import pymorphy2

tqdm.pandas()

In [42]:
morph = pymorphy2.MorphAnalyzer()
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

def review_to_wordlist(review, remove_stopwords=False):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    # достаем сам текст
    review_text = re.sub("[^а-яА-Я]"," ", review)
    # приводим к нижнему регистру и разбиваем на слова по символу пробела
    words = review_text.lower().split()
    
    if remove_stopwords:
      # убираем стоп-слова
        stops = stopwords.words("russian")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [91]:
df = pd.read_csv('data/hhparser_vacancy.csv')

In [31]:
def remove_tags(text: str) -> str:
    return re.sub('<[^>]*>', ' ', text)

In [110]:
df.description.fillna('none', inplace=True)

In [111]:
descriptions = df.description.progress_apply(remove_tags)

100%|██████████████████████████████████| 96225/96225 [00:02<00:00, 32189.84it/s]


In [112]:
from normalizer import normalize_sentences

In [113]:
normal_sents = normalize_sentences(descriptions)

100%|██████████████████████████████████| 96225/96225 [00:04<00:00, 20847.87it/s]


In [80]:
model = word2vec.Word2Vec(batch_words=200, window=4, min_count=5, workers=4, vector_size=200)

In [85]:
%%time
model.build_vocab(normal_sents)
model.train(normal_sents, total_examples=model.corpus_count, epochs=20)

CPU times: user 12min 17s, sys: 39.2 s, total: 12min 57s
Wall time: 8min


(263935203, 320459940)

In [86]:
model.wv.most_similar('компания')

[('холдинг', 0.6283978819847107),
 ('фирма', 0.5357980132102966),
 ('корпорация', 0.46684783697128296),
 ('банк', 0.44805485010147095),
 ('бренд', 0.43597322702407837),
 ('агентство', 0.43376657366752625),
 ('агрохолдинг', 0.4186793267726898),
 ('предприятие', 0.4169907867908478),
 ('ретейлер', 0.4165612459182739),
 ('концерн', 0.400160551071167)]

In [87]:
model.wv.most_similar('зарплата')

[('зп', 0.9215664267539978),
 ('доход', 0.7335813045501709),
 ('заработный', 0.7197492718696594),
 ('плата', 0.7189484238624573),
 ('вознаграждение', 0.6790640950202942),
 ('з', 0.661858081817627),
 ('оклад', 0.6359198093414307),
 ('заработок', 0.5794186592102051),
 ('оплата', 0.578693687915802),
 ('окладной', 0.4974970519542694)]

In [99]:
def text2vec(text):
    vector = np.array([.0 for _ in range(model.vector_size)])
    count = 0

    for word in text.split():
        if word in model.wv:
            vector += model.wv[word]
            count += 1

    if count != 0:
        vector /= count

    return vector


In [103]:
vectors = np.zeros((df.shape[0], model.vector_size))

for i, name in enumerate(df.name):
    vectors[i, :] = text2vec(name)

In [109]:
from sklearn.metrics.pairwise import cosine_similarity

In [145]:
def most_similar_vacancee(name, top_n=5):
    vector = text2vec(name)
    similarities = cosine_similarity([vector], vectors)
    similarities = [(sim, df.name.iloc[i]) for i, sim in enumerate(similarities[0])]
    similarities.sort(reverse=True)
    
    return similarities[:top_n]

In [149]:
most_similar_vacancee('разработка сайта')

[(1.0000000000000002, 'Стажер Sberseasons Мобильная разработка Android'),
 (1.0000000000000002,
  'Системный аналитик/ Бизнес-аналитик разработка высоконагруженного ПО'),
 (1.0000000000000002,
  'Разработчик С++, кроссплатформенная разработка сетевых сервисов'),
 (1.0000000000000002, 'Unity разработчик/Gamedev, разработка игр'),
 (1.0000000000000002, 'Product manager мобильная разработка')]

In [132]:
model.wv.most_similar('оператор')

[('диспетчер', 0.49753934144973755),
 ('колл', 0.4464859068393707),
 ('мерчендайзер', 0.4109043478965759),
 ('продавец', 0.39603227376937866),
 ('комплектовщик', 0.3930642306804657),
 ('телемаркетолог', 0.39092662930488586),
 ('волоконный', 0.3791011571884155),
 ('авторизированный', 0.3667648136615753),
 ('регистратор', 0.3634858727455139),
 ('сортировочный', 0.35270535945892334)]

In [88]:
model.save('hh_word2vec.pickle')